In [19]:
import os
import pandas as pd
import pickle
import shutil
import numpy as np
import seaborn as sns
from sklearn.datasets import load_files
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D, Flatten, Dense, Dropout, Input
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from keras.models import Model
from keras.layers.merge import concatenate

from PIL import ImageFile   
from sklearn.model_selection import train_test_split
from keras.preprocessing import image                  
from tqdm import tqdm

from keras.applications.inception_v3 import InceptionV3

# Defining the train,test and model directories

In [20]:
TEST_DIR = 'imgs/test'
TRAIN_DIR = 'imgs/train'
MODEL_PATH = 'model/GoogLeNet'
PICKLE_PATH = 'pickle'
CSV_DIR = 'csv_files'

In [21]:
if not os.path.exists(TEST_DIR):
    print("Testing data does not exists")
if not os.path.exists(TRAIN_DIR):
    print("Training data does not exists")
if not os.path.exists(MODEL_PATH):
    print("Model ResNet path does not exists")
    os.makedirs(MODEL_PATH)
    print("Model ResNet path created")
if not os.path.exists(PICKLE_PATH):
    os.makedirs(PICKLE_DIR)
if not os.path.exists(CSV_DIR):
    os.makedirs(CSV_DIR)# Data Preparation

# Data Preparation

In [26]:
data_train = pd.read_csv('csv_files/train.csv')
data_test = pd.read_csv('csv_files/test.csv')

### Converting into numerical values

In [27]:
labels_list = list(set(data_train['ClassName'].values.tolist()))
labels_id = {label_name:id for id,label_name in enumerate(labels_list)}
print(labels_id)
data_train['ClassName'].replace(labels_id,inplace=True)

{'c7': 0, 'c1': 1, 'c3': 2, 'c5': 3, 'c4': 4, 'c0': 5, 'c6': 6, 'c9': 7, 'c8': 8, 'c2': 9}


In [28]:
with open('pickle/labels_list.pkl', "wb") as handle:
    pickle.dump(labels_id,handle)

In [29]:
labels = to_categorical(data_train['ClassName'])
print(labels.shape)# Splitting into Train and Test sets

(22424, 10)


# Splitting into Train and Test sets

In [30]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest = train_test_split(data_train.iloc[:,0],labels,test_size = 0.2,random_state=42)

In [37]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(299, 299))
    # convert PIL.Image.Image type to 3D tensor with shape (64, 64, 3)
    x = image.img_to_array(img)
   # convert 3D tensor to 4D tensor with shape (1, 64, 64, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(xtrain).astype('float32')/255 - 0.5
valid_tensors = paths_to_tensor(xtest).astype('float32')/255 - 0.5

100%|████████████████████████████████████████████████████████████████████████████| 17939/17939 [07:01<00:00, 42.56it/s]


# GoogLeNet

In [35]:
model = InceptionV3(include_top=False)
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_195 (Conv2D)             (None, None, None, 3 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, None, None, 3 96          conv2d_195[0][0]                 
__________________________________________________________________________________________________
activation_94 (Activation)      (None, None, None, 3 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [36]:
train_googlenet = model.predict(train_tensors,verbose=1)
valid_googlenet = model.predict(valid_tensors,verbose=1)

ValueError: in user code:

    D:\Programas\anaconda3\lib\site-packages\keras\engine\training.py:1544 predict_function  *
        return step_function(self, iterator)
    D:\Programas\anaconda3\lib\site-packages\keras\engine\training.py:1527 run_step  *
        outputs = model.predict_step(data)
    D:\Programas\anaconda3\lib\site-packages\keras\engine\training.py:1500 predict_step  *
        return self(x, training=False)
    D:\Programas\anaconda3\lib\site-packages\keras\engine\base_layer.py:1006 __call__  *
        outputs = call_fn(inputs, *args, **kwargs)
    D:\Programas\anaconda3\lib\site-packages\keras\engine\functional.py:416 call  *
        inputs, training=training, mask=mask)
    D:\Programas\anaconda3\lib\site-packages\keras\engine\functional.py:551 _run_internal_graph  *
        outputs = node.layer(*args, **kwargs)
    D:\Programas\anaconda3\lib\site-packages\keras\engine\base_layer.py:1006 __call__  *
        outputs = call_fn(inputs, *args, **kwargs)
    D:\Programas\a